# Índice<a id="ind"></a>

[**Introdução**](#limp_ini)

[**Etapa 1**](#etapa1)
* [Análise de uma amostra: identificação tipos mínimos e transformações](#ana_amo)
    - [Identificação das transformações possíveis](#ident_trans)
    - [Identificação dos subtipos mínimos](#obt_tipo_min)       
        
[**Etapa 2**](#etapa2)
* [Carregamento e tratamento dos dados totais](#carr_tot)
* [Comparação entre dados com e sem optimização](#comp)

In [ ]:
# instala o pacote "optimizacaoDF"
!pip install git+https://github.com/joaogambaro/optimizacao_dataframes

In [1]:
import pandas as pd
import numpy as np
from pathlib import Path

import optimizacaoDF.coleta_amostra as amo
import optimizacaoDF.analise_resultados as ana
import optimizacaoDF.optimizacao_dataframe as opt

import os
import importlib
import warnings

pd.pandas.set_option('display.max_columns', None)
warnings.filterwarnings("ignore")

# Introdução<a id="limp_ini"></a>
[Índice](#ind)

O objetivo central deste notebook foi desenvolver uma rotina para abrir um arquivo ".csv" com um   consumo reduzido de memória.  A redução foi obtida identificando as colunas de dados que pode ser escritas com subtipos de variáveis menores.

    Exemplo: para os dados abaixo, a coluna "idade" foi carregada com o tipo `float64`, "nome" com o tipo `object` e "peso" com o tipo `float64`. Contudo, estes tipos necessitam de mais memória do que o necessário para o armazenamento dos dados em questão. Para estes dados,  por exemplo, poderíamos escrever as colunas com os tipo, int8 para "idade", "classify" para "nome" e float32 para "peso" sem perda de informação, reduzindo o uso de memória.

|idade(float64)|nome(object) | peso(float64)|
|--|--|--|
|34.0| João|64.5 |
|32.0| Camila|75.8 |
|31.0| José|65.3 |
|26.0| Ana|58.0 |
    
Este trabalho foi dividido em etapas

**Etapa 1: analisar um pequena amostra dos dados**

Nesta etapa é carregado em um dataframe com uma amostra dos dados completos. O dataframe contém somente algumas linhas do arquivo original, que podem ser escolhidas aleatoriamente ou em sequência. A amostra selecionada é usada para identificar os subtipos mínimos de cada coluna e as transformações que podem ser feitas.

**Etapa 2: carregamento dos dados totais**

Nesta etapa os dados são carregados utilizando o subtipos mínimos identificados na etapa 1. Depois do carregamento, as transformações identificadas na etapa 1 são aplicadas e, por final, uma nova identificação de subtipos é feita. Estes últimos passos  podem reduzir ainda mais o tamanho do dataframe  na memória. O resultado final é salvo em um novo arquivo.


**Observações**
- para auxiliar nos processo das etapa 1 e 2 foi criado o pacote "optimizacaoDF" que foi instalado no incio do notebook

- Quando os dados são salvos no final do processo, os subtipos que foram definidos não são salvos no arquivo. Ou seja, quando os dados forem abertos outras vezes, devemos novamente informar os tipos para reduzir o consumo de memória. Ainda assim, é importante salvar os o resultado final em um arquivo, pois muitos arquivos grandes possuem linhas em branco que serão eliminadas com o salvamento do resultado final. Além disso, as demais transformações feitas nos dados ficarão salvas com o novo arquivo.




# Etapa 1<a id="etapa1"></a>
[Índice](#ind)

## Análise de uma amostra: identificação tipos mínimos e transformações<a id="ana_amo"></a>

- Identificação das transformações possíveis:

Neste processo são identificadas algumas transformações possíveis. Como um exemplo, temos a coluna "price" que contém strings com valores de preço, (como "$500.00") que pode ter seus valores transformados para números.  A identificação é feita analisando individualmente cada coluna, e as funções das transformações são organizadas em um dicionário.
    

- Identificação dos subtipos mínimos: 

Este processo é feito usando o método `obtem_subtipos` do pacote `optimizacaoDF`, que retorna um dicionário com os nomes das colunas e os subtipos mínimos.


In [9]:
# caminho do arquivo com dados
current_path = os.getcwd()
file_name=Path(current_path)/"dados_originais/3_calendar.csv"

# coleta uma amostra(primeiras linhas)
tamAm=50000
df=amo.coleta_amostr(tamAm, file_name, is_rand_sample=False)

print("df.shape: ",df.shape)
display(df.head(3))

df.shape:  (50000, 7)


,listing_id,date,available,price,adjusted_price,minimum_nights,maximum_nights
0,53344884,2022-09-22,t,"$3,500.00","$3,500.00",3.0,365.0
1,53344884,2022-09-23,t,"$3,500.00","$3,500.00",3.0,365.0
2,53344884,2022-09-24,t,"$3,500.00","$3,500.00",3.0,365.0



### Identificação das transformações possíveis <a id="ident_trans"></a>

Colunas com transformações:


**price:** os dados originais são strings com preços ( exemplo: "$5,000.00"). Na transformação, as strings foram transformadas em números floats (para o exemplo temos 5000.00).

**available:** os dados originais são strigns com 'f' e 't' que indicam False ou True. Na transformação, 'f' e 't' viram respectivamente os inteiros 0 e 1.

**adjusted_price:** mesma transformação de "price" .



In [10]:
dict_transf={
    'price':
        lambda x: float(x.replace("$","").removesuffix(".00").replace(",",""))\
                  if not pd.isna(x)\
                  and x.replace("$","")\
                       .removesuffix(".00")\
                       .replace(",","").isdigit()\
                  else float('NaN'),
    
    'available':
        lambda x: 0 if x=='f' \
                    else (1 if x=='t' else float('NaN')),
    
    'adjusted_price':
        lambda x: float(x.replace("$","").removesuffix(".00").replace(",",""))\
                  if not pd.isna(x)\
                  and x.replace("$","")\
                       .removesuffix(".00")\
                       .replace(",","").isdigit()\
                  else float('NaN')
    
}

### Identificação dos subtipos mínimos<a id="obt_tipo_min"></a>
>**Observação:** em `obtem_subtipos` foi usado `"can_float_be_int=False"` para que colunas com float nunca sejam identificadas como int (se fosse usado True o número 23.00, por exemplo, poderia ser identificado como o int, 23). Como estamos lidando com uma amostra, a melhor opção é usar False para este parâmetro. Isto porque com True o método pode associar uma coluna float como int para a amostra, enquanto que nos dados totais pode ser que isto seja impossível. Este fato pode acontecer, por exemplo, quando quando na amostra não existem dados Nan (que são float), mas nos dados totais existem.

In [11]:
# obtem os subtipos minimos
dict_sub = opt.obtem_subtipos(df, can_float_be_int=False)
dict_sub

{'listing_id': 'uint64',
 'date': 'category',
 'available': 'category',
 'price': 'category',
 'adjusted_price': 'category',
 'minimum_nights': 'float32',
 'maximum_nights': 'float32'}

# Etapa 2<a id="etapa2"></a>
[Índice](#ind)

## Carregamento e tratamento dos dados totais<a id="carr_tot"></a>

Rotina seguida:
- 1. Carrega os dados com os subtipos mínimos identificados com a amostra

- 2. Aplica as transformações identificadas na amostra

- 3. Obtém novamente os subtipos mínimos para os dados

>**Observação:** Nesta etapa podemos encontrar novos subtipos mínimos, o que reduz ainda mais a memória usada pelo dataframe. Aqui, usamos o método `obtem_subtipos` com o parâmetro `"can_float_be_int=True"`, ou seja, agora tornamos possível números float serem identificados com int em situações adequadas. Ista escolha é feita porque o  método é aplicado nos dados completos e não em uma amostra.  Os novos subtipos que podem ocorrem devido a escolha do parâmetros 'can_float_be_int=True' e também devido aos dados transformados na etapa 2.

- 4. transforma os dados para os subtipos mínimos  da etapa 3 e salva os dados em um arquivo.


In [12]:
%%time

# 1. carrega os dados com os subtipos minimos
df_total=pd.read_csv(file_name,
                 low_memory=False,                 
                 dtype = dict_sub    #dicionário com tranformações
                )

#Obs: não foi necessário selecionar colunas pois todas
# as colunas serão carregadas


# uso de memória
mem_df_total = ana.uso_memoria(df_total)
print(f'memória do df_original: {round(mem_df_total,2)}Mb')

memória do df_original: 211.95Mb
CPU times: user 5.24 s, sys: 1.36 s, total: 6.61 s
Wall time: 8.08 s


In [13]:
%%time

# 2. aplica as transformações
opt.object_to_float(df_total,dict_transf,inplace=True)


# uso de memória
mem_df_total = ana.uso_memoria(df_total)
print(f'memória do df_original: {round(mem_df_total,2)}Mb')


memória do df_original: 385.48Mb
CPU times: user 189 ms, sys: 109 ms, total: 298 ms
Wall time: 352 ms


In [14]:
%%time

# 3. pesquisa novamente os subtipos mínimos
dict_sub = opt.obtem_subtipos(df_total, can_float_be_int=True)
dict_sub


CPU times: user 23.3 s, sys: 772 ms, total: 24.1 s
Wall time: 24.1 s


{'available': 'uint8', 'price': 'uint32', 'adjusted_price': 'uint32'}

In [15]:
# 4. transforma os dados para os subtipos mínimos 
opt.tranforma_tipos(df_total,dict_sub, inplace=True)

# uso de memória
mem_df_total = ana.uso_memoria(df_total)
print(f'memória do df_original: {round(mem_df_total,2)}Mb')

memória do df_original: 247.82Mb


In [16]:
%%time

# salva o resultado final
current_path = os.getcwd()
path_root=Path(current_path).parent
df_total.to_csv(path_root/"dados/3_calendar_opt.csv", index=False)

CPU times: user 26.3 s, sys: 486 ms, total: 26.8 s
Wall time: 29.7 s


## Verificação do resultado final

In [18]:
df_total.head(5)

,listing_id,date,available,price,adjusted_price,minimum_nights,maximum_nights
0,53344884,2022-09-22,1,3500,3500,3.0,365.0
1,53344884,2022-09-23,1,3500,3500,3.0,365.0
2,53344884,2022-09-24,1,3500,3500,3.0,365.0
3,53344884,2022-09-25,1,3500,3500,3.0,365.0
4,53344884,2022-09-26,1,3500,3500,3.0,365.0
